## Members with wikidata in relations without

```sql
WITH bounding_area AS (
    SELECT geometry FROM `bigquery-public-data.geo_openstreetmap.planet_features`
    WHERE feature_type="multipolygons"
    AND ('wikidata', 'Q218') IN (SELECT (key, value) FROM unnest(all_tags))
),
relations_wo_wikidata as (
    SELECT planet_relations.id, (SELECT value FROM unnest(planet_relations.all_tags) where key = 'name') as name, m.id as member_id
    FROM `bigquery-public-data.geo_openstreetmap.planet_relations` as planet_relations,
      planet_relations.members as m,
      bounding_area
    WHERE 'wikidata' NOT IN (SELECT key FROM UNNEST(all_tags))
    AND ST_DWithin(bounding_area.geometry, planet_relations.geometry, 0)
),
bounding_area_features AS (
    SELECT * FROM `bigquery-public-data.geo_openstreetmap.planet_features` as planet_features, bounding_area
    WHERE ST_DWithin(bounding_area.geometry, planet_features.geometry, 0)
)
SELECT relations_wo_wikidata.id as id, relations_wo_wikidata.name as name,
    ARRAY_AGG (STRUCT (planet_features.osm_id as id, (SELECT value FROM unnest(planet_features.all_tags) where key = 'name') as name, (SELECT value FROM unnest(planet_features.all_tags) where key = 'wikidata') as wikidata_id)) as members
FROM relations_wo_wikidata JOIN bounding_area_features as planet_features ON relations_wo_wikidata.member_id = planet_features.osm_id
WHERE 'wikidata' IN (SELECT key FROM UNNEST(all_tags))
GROUP BY id, name
```

## Points with wikidata in `multipolygons` without

```sql
WITH bounding_area AS (
    SELECT geometry FROM `bigquery-public-data.geo_openstreetmap.planet_features`
    WHERE feature_type="multipolygons"
    AND ('wikidata', 'Q218') IN (SELECT (key, value) FROM unnest(all_tags))
),
bounding_area_features AS (
    SELECT planet_features.osm_id, planet_features.feature_type, planet_features.geometry, planet_features.all_tags FROM `bigquery-public-data.geo_openstreetmap.planet_features` as planet_features, bounding_area
    WHERE ST_DWithin(bounding_area.geometry, planet_features.geometry, 0)
),
polygons_wo_wikidata as (
    SELECT planet_multipolygons.osm_id as id, (SELECT value FROM unnest(planet_multipolygons.all_tags) where key = 'name') as name, planet_multipolygons.geometry as geometry
    FROM bounding_area_features as planet_multipolygons,
      bounding_area
    WHERE feature_type = 'multipolygons'
    AND osm_id IS NOT NULL
    AND 'wikidata' NOT IN (SELECT key FROM UNNEST(all_tags))
    AND ST_DWithin(bounding_area.geometry, planet_multipolygons.geometry, 0)
)
SELECT polygons_wo_wikidata.id as id, polygons_wo_wikidata.name as name,
    ARRAY_AGG (STRUCT (baf.osm_id as id, (SELECT value FROM unnest(baf.all_tags) where key = 'name') as name, (SELECT value FROM unnest(baf.all_tags) where key = 'wikidata') as wikidata_id)) as points
FROM bounding_area_features as baf, polygons_wo_wikidata
WHERE 'wikidata' IN (SELECT key FROM UNNEST(all_tags))
AND baf.feature_type = "points"
AND ST_DWithin(baf.geometry, polygons_wo_wikidata.geometry, 0)
GROUP BY 1,2
```

Doing a fuzzy match between point and bounding polygon name we can filter the list and identify objects that might reffer to the same thing.

In [49]:
import pandas as pd

from fuzzywuzzy import fuzz
from unidecode import unidecode


pg_pt_df = pd.read_csv('data/wikidata/polygons_wo_points_with.csv')
pg_pt_df['score'] = pg_pt_df.apply(lambda row: fuzz.token_sort_ratio(unidecode(str(row['polygon_name'])), unidecode(str(row['point_name']))), axis=1)
display(pg_pt_df.shape[0])

# filter rows with score > 90 or NaN polygon_name and remove the score column
pg_pt_df = pg_pt_df[(pg_pt_df['score'] > 90) | pg_pt_df['polygon_name'].isna() | pg_pt_df['point_name'].isna()].drop(columns=['score'])
display(pg_pt_df.shape[0])
pg_pt_df.set_index('polygon_id', inplace=True)
pg_pt_df.index.astype(int)
pg_pt_df.sort_index(inplace=True, ascending=True)
pg_pt_df.head(20)


22995

2889

,polygon_name,point_id,point_name,point_wikidata_id
polygon_id,,,,
1207838,Iași,129984582,Iași,Q46852
1256013,NaN,463189475,Băcălești,Q12081691
1256791,NaN,463259349,Zărneni,Q12106654
1258022,NaN,463180837,Valea Largă,Q12087478
1258022,NaN,463180863,Valea Mantei,Q12087496
1258022,NaN,462259433,Arioneștii Noi,Q12079410
1258022,NaN,462259590,Valea Mieilor,Q12087495
1261720,NaN,447930799,Vispești,Q12093495
1261727,NaN,463196663,Pietroasa,Q12137214


In [50]:
pg_pt_df.to_csv('data/wikidata/polygons_wo_points_with_filtered.csv')